In [37]:
import tensorflow as tf
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
import time

In [38]:

from xgboost import XGBClassifier
from xgboost import XGBRegressor

## Data Pedestrian

In [39]:
train = np.loadtxt('/work/users/d/d/ddinh/aaco/input_data/MelbournePedestrian/MelbournePedestrian_TRAIN.txt')
Xtrn = train[:, 1:]
Ytrn = train[:, 0]
valid = np.loadtxt('/work/users/d/d/ddinh/aaco/input_data/MelbournePedestrian/MelbournePedestrian_TEST.txt')
Xval = valid[:, 1:]
Yval = valid[:, 0]

Ytrn = np.eye(11)[Ytrn.astype(int)]
Yval = np.eye(11)[Yval.astype(int)]
Ytrn = Ytrn[:, 1:]
Yval = Yval[:, 1:]

In [40]:
masksper = 256
d = Xtrn.shape[1]
X_class = np.concatenate([Xtrn]*masksper, 0)
Y_class = np.concatenate([Ytrn]*masksper, 0)
B = np.concatenate(
[np.sum(np.random.permutation(np.eye(d))[:, :np.random.randint(d)], 1, keepdims=True) for _ in range(X_class.shape[0])],
1)
B = np.float32(B.T)
X_class = np.concatenate((X_class*B, B), 1)
# Train classifier
est = XGBClassifier(n_estimators=256, device='gpu')
est.fit(X_class, Y_class)

/work/users/d/d/ddinh/.venv/lib64/python3.11/site-packages/xgboost/core.py:158: UserWarning: [11:38:03] WARNING: /workspace/src/context.cc:43: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
/work/users/d/d/ddinh/.venv/lib64/python3.11/site-packages/xgboost/core.py:158: UserWarning: [11:38:03] WARNING: /workspace/src/context.cc:196: XGBoost is not compiled with CUDA support.
  warnings.warn(smsg, UserWarning)


KeyboardInterrupt: 

In [63]:
# save model to path 
path = '/work/users/d/d/ddinh/aaco/models/'
est.save_model(path + 'pedestrian.model')

/work/users/d/d/ddinh/.venv/lib64/python3.11/site-packages/xgboost/core.py:158: UserWarning: [22:55:52] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


In [27]:
Bval = np.concatenate(
  [np.sum(np.random.permutation(np.eye(d))[:, :np.random.randint(d)], 1, keepdims=True) for _ in range(Xval.shape[0])],
  1)
Bval = np.float32(Bval.T)
Xvalmasked = np.concatenate((Xval*Bval, Bval), 1)

In [56]:
val_preds = est.predict_proba(np.concatenate((Xval, np.ones_like(Xval)), 1))

In [60]:
print(np.mean(np.round(val_preds)==Yval))

0.9877408774087741


In [61]:
val_preds = est.predict_proba(Xvalmasked) 

In [62]:
print(np.mean(np.round(val_preds)==Yval))

0.9692496924969249


# DATA Digits

In [1]:

from tslearn.datasets import UCR_UEA_datasets
data_loader = UCR_UEA_datasets()

Xtrn, Ytrn, Xval, Yval = data_loader.load_dataset(
        "PenDigits")
Ytrn = np.eye(10)[Ytrn.astype(int)]
Yval = np.eye(10)[Yval.astype(int)]

NameError: name 'np' is not defined

In [35]:
Ytrn[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])

In [36]:
Xtrn.shape

(7494, 8, 2)

In [37]:
masksper = 2
d = Xtrn.shape[1]
X_class = np.concatenate([Xtrn]*masksper, 0)
Y_class = np.concatenate([Ytrn]*masksper, 0)
B = np.concatenate(
[np.sum(np.random.permutation(np.eye(d))[:, :np.random.randint(d)], 1, keepdims=True) for _ in range(X_class.shape[0])],
1)
B = np.float32(B.T)

In [38]:
B = np.expand_dims(B, -1)

In [39]:
B = np.repeat(B, X_class.shape[-1], axis=-1)

In [40]:
B.shape

(14988, 8, 2)

In [41]:
X_class = np.concatenate((X_class*B, B), -1)
# Train classifier
est = XGBClassifier(n_estimators=256)
est.fit(X_class, Y_class)

ValueError: Please reshape the input data into 2-dimensional matrix.

In [ ]:
X_class = np.concatenate((X_class*B, B), 1)
# Train classifier
est = XGBClassifier(n_estimators=256)
est.fit(X_class, Y_class)

# Paper

In [1]:
import tensorflow as tf
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
import time
from xgboost import XGBClassifier
from xgboost import XGBRegressor
import sys
sys.path.append('/work/users/d/d/ddinh/aaco/src')
from load_dataset import load_adni_data

2024-10-28 13:34:48.849882: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-28 13:34:48.869680: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-28 13:34:48.875016: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-28 13:34:48.943867: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-28 13:34:52.849958: W tensorflow/compiler/tf2

In [3]:

dataset = load_adni_data()
x = dataset.x
y = dataset.y

mask_nan = np.isnan(x)

x[mask_nan] = 0

In [6]:
mask_nan = np.isnan(x)
# get the number of nan values
total = mask_nan.sum()
probability = total / x.size
probability

0.40976380572188953

In [3]:
x_all = []
for item in x:
    x_all.append(item.flatten('F'))
x_all = np.array(x_all)

In [4]:
# split dataset into train and test sets 80/20 with random seed 42
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_all, y, test_size=0.2, random_state=42)

In [5]:
y_val.shape

(201, 12, 3)

In [6]:
masksper = 256
d = x_train.shape[1]
X_class = np.concatenate([x_train]*masksper, 0)
Y_class = np.concatenate([y_train]*masksper, 0)
B = np.concatenate(
[np.sum(np.random.permutation(np.eye(d))[:, :np.random.randint(d)], 1, keepdims=True) for _ in range(X_class.shape[0])],
1)
B = np.float32(B.T)

# remove 0 mask
mask_nonzero = np.sum(B, axis=1) != 0
B = B[mask_nonzero]
X_class = X_class[mask_nonzero]
Y_class = Y_class[mask_nonzero]


In [7]:

mask_nan = X_class == 0
B[mask_nan] = 0

In [8]:
X_class = np.concatenate((X_class*B, B), 1)

In [9]:
num_ts = y_train.shape[1]

location = []
for i in range(B.shape[0]):
    b = B[i]
    temp = -1
    for j in range(4):
        part = b[num_ts*j:num_ts*(j+1)]
        max_index = np.where(part == 1)[0]
        if len(max_index) > 0:
            max_index = max_index[-1]
            if max_index > temp:
                temp = max_index
    location.append(temp)
    
location = np.array(location)

In [10]:
location.shape

(200790,)

In [11]:
np.isnan(np.array([1,2,3,np.nan,5]))

array([False, False, False,  True, False])

In [12]:
y_temp = []
for i in range(Y_class.shape[0]):
    y_location = Y_class[i, location[i]]
    mask_nan = np.isnan(y_location)
    y_location[mask_nan] = 0
    y_temp.append(y_location)
y_temp = np.array(y_temp)

In [13]:
mask_zero = np.sum(y_temp, axis=1) != 0
X_class = X_class[mask_zero]
Y_class = y_temp[mask_zero]

In [14]:
mask_nonzero_y = np.sum(y_temp, axis=1) != 0

In [15]:
y_temp[ np.sum(y_temp, axis=1) == 0].shape

(3424, 3)

In [16]:
Y_class = np.argmax(Y_class, axis=1)

In [17]:
# Train classifier
est = XGBClassifier(n_estimators=256)
est.fit(X_class, Y_class)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=256, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [19]:
X_class[0]

array([ 0.3296681 ,  0.3296681 ,  0.        ,  0.        ,  0.3296681 ,
        0.        ,  0.3296681 ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.13901885,  0.        ,  0.13901885,
        0.        ,  0.        ,  0.        ,  0.03456135,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.0882828 ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , -0.        ,  0.        , -0.18216652, -0.        ,
        0.24137278,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  1.        ,  1.        ,
        0.        ,  0.        ,  1.        ,  0.        ,  1.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  1.        ,  0.        ,  0.        ,
        0.        ,  1.        ,  0.        ,  0.        ,  0.  

In [20]:
est.predict_proba([X_class[0]])

array([[0.00643092, 0.6626856 , 0.3308835 ]], dtype=float32)

In [33]:
est.save_model('/work/users/d/d/ddinh/aaco/models/adni_part2.model')

/work/users/d/d/ddinh/.venv/lib64/python3.11/site-packages/xgboost/core.py:158: UserWarning: [11:44:17] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


In [21]:
Bval = np.concatenate(
  [np.sum(np.random.permutation(np.eye(d))[:, :np.random.randint(d)], 1, keepdims=True) for _ in range(x_val.shape[0])],
  1)
Bval = np.float32(Bval.T)

mask_nonzero = np.sum(Bval, axis=1) != 0
Bval = Bval[mask_nonzero]
Xval = x_val[mask_nonzero]
Yval = y_val[mask_nonzero]

In [22]:
mask_nan = np.isnan(Xval)
Bval[mask_nan] = 0

In [23]:
Xvalmasked = np.concatenate((Xval*Bval, Bval), 1)

In [24]:
num_ts = y_val.shape[1]

location = []
for i in range(Bval.shape[0]):
    b = Bval[i]
    temp = -1
    for j in range(4):
        part = b[num_ts*j:num_ts*(j+1)]
        max_index = np.where(part == 1)[0]
        if len(max_index) > 0:
            max_index = max_index[-1]
            if max_index > temp:
                temp = max_index
    location.append(temp)
location = np.array(location)

In [25]:
y_temp = []
for i in range(Yval.shape[0]):
    y_location = Yval[i, location[i]]
    mask_nan = np.isnan(y_location)
    y_location[mask_nan] = 0
    y_temp.append(y_location)

In [26]:
mask_zero = np.sum(y_temp, axis=1) != 0
Xvalmasked = Xvalmasked[mask_zero]
Yval = np.array(y_temp)[mask_zero]

In [27]:
val_preds = est.predict_proba(Xvalmasked) 

In [29]:
print(np.mean(np.argmax(val_preds, 1)==np.argmax(Yval, 1)))

0.5128205128205128


In [24]:
from sklearn.metrics import roc_curve, auc
import numpy as np

n_classes = 3

# Dictionary to store fpr, tpr, and roc_auc for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

# Calculate ROC curve and AUC for each class
for i in range(n_classes):
    # y_true[:, i] gives the binary labels for class `i`
    # y_pred[:, i] gives the predicted probabilities for class `i`
    fpr[i], tpr[i], _ = roc_curve(Yval[:, i], np.round(val_preds)[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Display the results
for i in range(n_classes):
    print(f"Class {i}: AUC = {roc_auc[i]:.2f}")

# average: 
print(f"Average: AUC = {np.mean(list(roc_auc.values())):.2f}")

Class 0: AUC = 0.76
Class 1: AUC = 0.54
Class 2: AUC = 0.64
Average: AUC = 0.65


In [27]:
import torch

a = torch.Tensor([1,2,3,4,5]) 

In [29]:
a[a==1] = 0

In [30]:
a

tensor([0., 2., 3., 4., 5.])

In [ ]:
x